http://surpriselib.com

<h1 style="color:blue; font-weight:bold">Assignment todo list</h1>

- Extract all the pipeline implemented function in a python script and import them in the notebook to use them.
- Implement the *get_user_recommendation* function.
- **Bonus** : Generalize the *get_trained_model* function to use any surprise model kwargs.

# Load data

<p style="color:blue; font-weight:bold">TODO</p>

- Extract the implemened functions in a python script and import them in the notebook.

In [1]:
import sys
sys.path.insert(0, '../recsys_dsa_20/')
from pipeline import *

from pathlib import Path
ratings_filepath = Path('../data/movielens/ml-latest-small/ratings.csv')

## From surprise

In [2]:
ratings = load_ratings_from_surprise()
ratings

## From file

In [3]:
ratings = load_ratings_from_file(ratings_filepath)
ratings

## Modular function

In [4]:
data = get_data(from_surprise=True)
data

# Manual pipeline

<p style="color:blue; font-weight:bold">TODO</p>

- Extract the implemened functions in a python script and import them in the notebook.

## Split data in train and test

In [5]:
from surprise.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
train

In [6]:
train.n_users, train.n_items

(943, 1651)

## Train model

<p style="color:blue; font-weight:bold">TODO</p>

- Change the *model_kwargs* argument in the *get_trained_model* function to make it usable for any surprise model (SVD, KNN, NMF, etc).

In [7]:
from surprise import SVD

model = SVD()

In [8]:
model.fit(train)

## Make predictions

In [9]:
predictions = model.test(test)
predictions[:10]

[Prediction(uid='907', iid='143', r_ui=5.0, est=4.943397808530095, details={'was_impossible': False}),
 Prediction(uid='371', iid='210', r_ui=4.0, est=3.8593830243710396, details={'was_impossible': False}),
 Prediction(uid='218', iid='42', r_ui=4.0, est=3.4183924083452157, details={'was_impossible': False}),
 Prediction(uid='829', iid='170', r_ui=4.0, est=3.945166269173857, details={'was_impossible': False}),
 Prediction(uid='733', iid='277', r_ui=1.0, est=3.0782845100343827, details={'was_impossible': False}),
 Prediction(uid='363', iid='1512', r_ui=1.0, est=3.3300895558579184, details={'was_impossible': False}),
 Prediction(uid='193', iid='487', r_ui=5.0, est=3.7178383140392244, details={'was_impossible': False}),
 Prediction(uid='808', iid='313', r_ui=5.0, est=4.574221931381935, details={'was_impossible': False}),
 Prediction(uid='557', iid='682', r_ui=2.0, est=3.740851670552038, details={'was_impossible': False}),
 Prediction(uid='774', iid='196', r_ui=3.0, est=2.07464171937463, de

## Evaluation

In [10]:
from surprise import accuracy

accuracy.rmse(predictions=predictions)

RMSE: 0.9360


0.9360138039171689

In [11]:
accuracy.mae(predictions=predictions)

MAE:  0.7378


0.7377851540448496

## Modular code

In [12]:
model_kwargs = {'sim_options': {'user_based': True, 'name': 'pearson'}}

my_model, metrics_dict = train_and_evalute_model_pipeline(KNNBasic, model_kwargs)
metrics_dict

get_trained the model <class 'surprise.prediction_algorithms.knns.KNNBasic'> with model_kwargs: {'sim_options': {'user_based': True, 'name': 'pearson'}}.
Computing the pearson similarity matrix...
Done computing similarity matrix.


{'RMSE': 1.0150350905205965, 'MAE': 1.0150350905205965}

In [14]:
my_model, metrics_dict = train_and_evalute_model_pipeline(SVD)
metrics_dict

get_trained the model <class 'surprise.prediction_algorithms.matrix_factorization.SVD'> without model_kwargs.


{'RMSE': 0.9363352025038504, 'MAE': 0.9363352025038504}

In [15]:
get_trained_model(KNNBasic, train, {'sim_options':{'user_based': False, 'name': 'pearson'}})

get_trained the model <class 'surprise.prediction_algorithms.knns.KNNBasic'> with model_kwargs: {'sim_options': {'user_based': False, 'name': 'pearson'}}.
Computing the pearson similarity matrix...
Done computing similarity matrix.


# Benchmarking

<p style="color:blue; font-weight:bold">TODO</p>

- Add the other models (baseline, item based with cosine & pearson sim metrics, NMF, SVD)
- Add the fit_time in the benchmarking

In [ ]:
from surprise import BaselineOnly, KNNBasic, NMF, SVD, SVDpp
import time
benchmark_dict = {}

model_dict_list = [
    {
        'model_name' : 'BaselineOnly',
        'model_class' : BaselineOnly,
        'model_kwargs' : {}
    },
    {
        'model_name' : 'KNN user based with cosine similarity',
        'model_class' : KNNBasic,
        'model_kwargs' : {'sim_options': {'user_based': True, 'name': 'cosine'}}
    },
    {
        'model_name' : 'KNN user based with pearson similarity',
        'model_class' : KNNBasic,
        'model_kwargs' : {'sim_options': {'user_based': True, 'name': 'pearson'}}
    },
    {
        'model_name' : 'KNN item based with cosine similarity',
        'model_class' : KNNBasic,
        'model_kwargs' : {'sim_options': {'user_based': False, 'name': 'cosine'}}
    },
    {
        'model_name' : 'KNN item based with pearson similarity',
        'model_class' : KNNBasic,
        'model_kwargs' : {'sim_options': {'user_based': False, 'name': 'pearson'}}
    },
    {
        'model_name' : 'SVD',
        'model_class' : SVD,
        'model_kwargs' : {}
    },
    {
        'model_name' : 'SVD++',
        'model_class' : SVDpp,
        'model_kwargs' : {}
    },
    {
        'model_name' : 'NMF',
        'model_class' : NMF,
        'model_kwargs' : {}
    }
]
starts = time.time()   
for model_dict in model_dict_list:
    now = time.time()

    model, metrics_dict = train_and_evalute_model_pipeline(
        model_dict['model_class'], model_dict['model_kwargs'])
    metrics_dict['train_and_evaluate_time'] = now - starts
    benchmark_dict[model_dict['model_name']] = metrics_dict
    model_dict['fitted_model'] = model    
    print("It has been {0} seconds since the loop started".format(now - program_starts))
    starts = now

benchmark_dict

get_trained the model <class 'surprise.prediction_algorithms.baseline_only.BaselineOnly'> without model_kwargs.
Estimating biases using als...
It has been 196.02118110656738 seconds since the loop started
get_trained the model <class 'surprise.prediction_algorithms.knns.KNNBasic'> with model_kwargs: {'sim_options': {'user_based': True, 'name': 'cosine'}}.
Computing the cosine similarity matrix...
Done computing similarity matrix.
It has been 196.5559275150299 seconds since the loop started
get_trained the model <class 'surprise.prediction_algorithms.knns.KNNBasic'> with model_kwargs: {'sim_options': {'user_based': True, 'name': 'pearson'}}.
Computing the pearson similarity matrix...
Done computing similarity matrix.
It has been 200.6057367324829 seconds since the loop started
get_trained the model <class 'surprise.prediction_algorithms.knns.KNNBasic'> with model_kwargs: {'sim_options': {'user_based': False, 'name': 'cosine'}}.
Computing the cosine similarity matrix...
Done computing si

In [ ]:
# Save in pandas for the plotting
import pandas as pd
df = pd.DataFrame(benchmark_dict)
df.bar()

# Cross validation

In [ ]:
from surprise.model_selection import cross_validate

cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# User recommendation

<p style="color:blue; font-weight:bold">TODO</p>

- Create a function that 

In [ ]:
import pandas

def get_user_recommendation(model: AlgoBase, user_id: int, k: int, data, movies : pandas.DataFrame
                           ) -> pandas.DataFrame:
    """Makes movie recommendations a user.
    
    Parameters
    ----------
        model : AlgoBase
            A trained surprise model
        user_id : int
            The user for whom the recommendation will be done.
        k : int
            The number of items to recommend.
        data : FIXME
            The data needed to do the recommendation.
        movies : pandas.DataFrame
            The dataframe containing the movies metadata (title, genre, etc)
        
    Returns
    -------
    pandas.Dataframe
        A dataframe with the k movies that will be recommended the user. The dataframe should have the following
        columns (movie_name : str, movie_genre : str, predicted_rating : float, true_rating : float)
        
    Notes
    -----
    - You should create other functions that are used in this one and not put all the code in the same function.
        For example to create the final dataframe, instead of implemented all the code
        in this function (get_user_recommendation), you can create a new one (create_recommendation_dataframe)
        that will be called in this function.
    - You can add other arguments to the function if you need to.
    """
    # FIXME
    pass